In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
from gtsam.symbol_shorthand import X
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import inv

from lac.utils.frames import get_cam_pose_rover, CAMERA_TO_OPENCV_PASSIVE
from lac.perception.vision import get_camera_intrinsics
from lac.slam.slam import SLAM
from lac.slam.visual_odometry import StereoVisualOdometry
from lac.slam.feature_tracker import FeatureTracker
from lac.utils.plotting import plot_poses, plot_surface, plot_3d_points
from lac.util import load_data, load_stereo_images, load_side_images
from lac.params import CAMERA_INTRINSICS
from lac.utils.frames import (
    apply_transform,
    invert_transform_mat,
    OPENCV_TO_CAMERA_PASSIVE,
    get_cam_pose_rover,
)
from lac.params import LAC_BASE_PATH

print(LAC_BASE_PATH)
%load_ext autoreload
%autoreload 2

In [ ]:
# Load the data logs
data_path = "/home/shared/data_raw/LAC/segmentation/slam_map1_preset1_teleop"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")
# Load the images
left_imgs, right_imgs = load_stereo_images(data_path)
side_left_imgs, side_right_imgs = load_side_images(data_path)

In [ ]:
# Load the ground truth map
map = np.load(
    "/home/shared/data_raw/LAC/heightmaps/competition/Moon_Map_01_preset_0.dat",
    allow_pickle=True,
)

In [ ]:
START_FRAME = 100

IMG_RATE = 2
KEYFRAME_RATE = 10
GRAPH_UPDATE_RATE = 2
GRAPH_OPTIMIZE_RATE = 1000
END_FRAME = 5654

curr_pose = initial_pose
step = START_FRAME
prev_step = None
prev_pose = None

In [ ]:
# Track
svo = StereoVisualOdometry(cam_config)
svo.initialize(initial_pose, left_imgs[START_FRAME], right_imgs[START_FRAME])

prev_step = step
step = START_FRAME + IMG_RATE

svo.track(left_imgs[step], right_imgs[step])

prev_pose = curr_pose
curr_pose = svo.get_pose()

In [ ]:
tracker = FeatureTracker(cam_config)

cam_name = "Left"

if cam_name == "Right":
    prev_img = side_right_imgs[prev_step]
    next_img = side_right_imgs[step]
else:
    prev_img = side_left_imgs[prev_step]
    next_img = side_left_imgs[step]

prev_feats = tracker.extract_feats(prev_img)
next_feats = tracker.extract_feats(next_img)

matches = tracker.match_feats(prev_feats, next_feats, min_score=0.9)
points_prev = prev_feats["keypoints"][0][matches[:, 0]]
points_next = next_feats["keypoints"][0][matches[:, 1]]

points2d_prev = points_prev.cpu().numpy()
points2d_next = points_next.cpu().numpy()

# Camera intrinsics and extrinsics
K = get_camera_intrinsics(cam_name, cam_config)
rover_T_cam = get_cam_pose_rover(cam_name)
rover_T_cam_ocv = rover_T_cam.copy()
rover_T_cam_ocv[:3, :3] = rover_T_cam_ocv[:3, :3] @ CAMERA_TO_OPENCV_PASSIVE

# Projection matrices
rover_poses = [curr_pose]
for _ in range(10):
    cam_T_world_0 = inv(prev_pose @ rover_T_cam_ocv)
    cam_T_world_1 = inv(rover_poses[-1] @ rover_T_cam_ocv)

    P0 = K @ cam_T_world_0[:3]
    P1 = K @ cam_T_world_1[:3]

    MAX_DEPTH = 10.0

    points_4d_h = cv2.triangulatePoints(P1, P0, points2d_next.T, points2d_prev.T)
    points_3d_next = (points_4d_h[:3] / points_4d_h[3]).T
    depths_next = (cam_T_world_1[:3, :3] @ points_3d_next.T + cam_T_world_1[:3, 3:4]).T[:, 2]
    depth_mask = (depths_next > 0) & (depths_next < MAX_DEPTH)

    # Initial guess
    rover_to_cam = get_cam_pose_rover(cam_name)
    cam_to_rover = invert_transform_mat(rover_to_cam)
    w_T_c = rover_pose @ rover_to_cam
    w_T_c[:3, :3] = w_T_c[:3, :3] @ np.linalg.inv(OPENCV_TO_CAMERA_PASSIVE)
    est_pose = invert_transform_mat(w_T_c)
    R = est_pose[:3, :3]
    tvec2 = est_pose[:3, 3].reshape(3, 1)
    rvec2, _ = cv2.Rodrigues(R)

    success, rvec, tvec, inliers = cv2.solvePnPRansac(
        objectPoints=points_3d_next[depth_mask],
        imagePoints=points2d_next[depth_mask],
        cameraMatrix=CAMERA_INTRINSICS,
        distCoeffs=None,
        flags=cv2.SOLVEPNP_ITERATIVE,
        reprojectionError=0.1,
        iterationsCount=100,
        useExtrinsicGuess=True,
        rvec=rvec2,
        tvec=tvec2,
    )

    if success:
        # TODO: clean up this code
        R, _ = cv2.Rodrigues(rvec)
        T = np.hstack((R, tvec))
        est_pose = np.vstack((T, [0, 0, 0, 1]))
        w_T_c = invert_transform_mat(est_pose)
        w_T_c[:3, :3] = w_T_c[:3, :3] @ OPENCV_TO_CAMERA_PASSIVE
        rover_to_cam = get_cam_pose_rover(cam_name)
        cam_to_rover = invert_transform_mat(rover_to_cam)
        rover_pose = w_T_c @ cam_to_rover

    rover_poses.append(rover_pose)

In [ ]:
t_errors = []
for i in range(len(rover_poses)):
    t_errors.append(np.linalg.norm(rover_poses[i][:3, 3] - poses[step][:3, 3]))

t_errors = np.array(t_errors)
plt.plot(t_errors * 100)
plt.xlabel("Iteration")
plt.ylabel("Translation Error (cm)")
print("Initial error", t_errors[0])
print("Final error", t_errors[-1])
print("Improvement", t_errors[0] - t_errors[-1])

In [ ]:
plt.scatter(points2d_next[:, 1], depths_next)

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(next_img, cmap="gray")
for i in range(len(points2d_prev)):
    if not depth_mask[i]:
        continue
    plt.plot([points2d_prev[i, 0], points2d_next[i, 0]], [points2d_prev[i, 1], points2d_next[i, 1]], color="lime")
    x, y = points2d_next[i]
    plt.scatter(x, y, color="lime", s=5)
    plt.text(x + 2, y, f"{depths_next[i]:.2f}", color="red", fontsize=8)
plt.axis("off")
plt.show()

In [ ]:
svo_poses = [initial_pose]
curr_pose = initial_pose

END_FRAME = 5654
END_FRAME = 1000

svo = StereoVisualOdometry(cam_config)
svo.initialize(initial_pose, left_imgs[START_FRAME], right_imgs[START_FRAME])

# Main loop over image frames
for curr_step in tqdm(range(START_FRAME + IMG_RATE, END_FRAME + 1, IMG_RATE)):
    if curr_step >= max(left_imgs.keys()):
        print("Reached end of images")
        break
    # Run VO for real-time pose tracking
    svo.track(left_imgs[curr_step], right_imgs[curr_step])
    curr_pose = svo.get_pose()
    svo_poses.append(curr_pose)

In [ ]:
def update_pose_mono(cam_name, prev_img, next_img, curr_pose, n_iter=20):

    new_pose = curr_pose.copy()

    prev_feats = tracker.extract_feats(prev_img)
    next_feats = tracker.extract_feats(next_img)

    matches = tracker.match_feats(prev_feats, next_feats, min_score=0.9)
    points_prev = prev_feats["keypoints"][0][matches[:, 0]]
    points_next = next_feats["keypoints"][0][matches[:, 1]]

    points2d_prev = points_prev.cpu().numpy()
    points2d_next = points_next.cpu().numpy()

    # Camera intrinsics and extrinsics
    K = get_camera_intrinsics(cam_name, cam_config)
    rover_T_cam = get_cam_pose_rover(cam_name)
    rover_T_cam_ocv = rover_T_cam.copy()
    rover_T_cam_ocv[:3, :3] = rover_T_cam_ocv[:3, :3] @ CAMERA_TO_OPENCV_PASSIVE

    # Projection matrices
    for it in range(n_iter):
        cam_T_world_0 = inv(prev_pose @ rover_T_cam_ocv)
        cam_T_world_1 = inv(new_pose @ rover_T_cam_ocv)

        P0 = K @ cam_T_world_0[:3]
        P1 = K @ cam_T_world_1[:3]

        MAX_DEPTH = 15.0

        points_4d_h = cv2.triangulatePoints(P1, P0, points2d_next.T, points2d_prev.T)
        points_3d_next = (points_4d_h[:3] / points_4d_h[3]).T
        depths_next = (cam_T_world_1[:3, :3] @ points_3d_next.T + cam_T_world_1[:3, 3:4]).T[:, 2]
        depth_mask = (depths_next > 0) & (depths_next < MAX_DEPTH)

        if depth_mask.sum() < 20:
            print(f"Not enough points in depth mask {depth_mask.sum()}, iteration {it}")
            break

        # Initial guess
        rover_to_cam = get_cam_pose_rover(cam_name)
        cam_to_rover = invert_transform_mat(rover_to_cam)
        w_T_c = new_pose @ rover_to_cam
        w_T_c[:3, :3] = w_T_c[:3, :3] @ np.linalg.inv(OPENCV_TO_CAMERA_PASSIVE)
        est_pose = invert_transform_mat(w_T_c)
        R = est_pose[:3, :3]
        tvec2 = est_pose[:3, 3].reshape(3, 1)
        rvec2, _ = cv2.Rodrigues(R)

        success, rvec, tvec, inliers = cv2.solvePnPRansac(
            objectPoints=points_3d_next[depth_mask],
            imagePoints=points2d_next[depth_mask],
            cameraMatrix=CAMERA_INTRINSICS,
            distCoeffs=None,
            flags=cv2.SOLVEPNP_ITERATIVE,
            reprojectionError=0.1,
            iterationsCount=100,
            useExtrinsicGuess=True,
            rvec=rvec2,
            tvec=tvec2,
        )

        if success:
            # TODO: clean up this code
            R, _ = cv2.Rodrigues(rvec)
            T = np.hstack((R, tvec))
            est_pose = np.vstack((T, [0, 0, 0, 1]))
            w_T_c = invert_transform_mat(est_pose)
            w_T_c[:3, :3] = w_T_c[:3, :3] @ OPENCV_TO_CAMERA_PASSIVE
            rover_to_cam = get_cam_pose_rover(cam_name)
            cam_to_rover = invert_transform_mat(rover_to_cam)
            new_pose = w_T_c @ cam_to_rover
        else:
            print("PnP failed")
            break

    return new_pose

In [ ]:
iter_svo_poses = [initial_pose]

END_FRAME = 5654
END_FRAME = 1000

curr_pose = initial_pose
curr_step = START_FRAME
prev_step = curr_step
prev_pose = curr_pose

svo = StereoVisualOdometry(cam_config)
svo.initialize(initial_pose, left_imgs[START_FRAME], right_imgs[START_FRAME])

# Main loop over image frames
for curr_step in tqdm(range(START_FRAME + IMG_RATE, END_FRAME + 1, IMG_RATE)):
    if curr_step >= max(left_imgs.keys()):
        print("Reached end of images")
        break

    # Run VO for real-time pose tracking
    svo.track(left_imgs[curr_step], right_imgs[curr_step])
    curr_pose = svo.get_pose()

    # Update
    prev_img_right = side_right_imgs[prev_step]
    next_img_right = side_right_imgs[curr_step]
    curr_pose = update_pose_mono("Right", prev_img_right, next_img_right, curr_pose, n_iter=20)

    prev_img_left = side_left_imgs[prev_step]
    next_img_left = side_left_imgs[curr_step]
    curr_pose = update_pose_mono("Left", prev_img_left, next_img_left, curr_pose, n_iter=20)

    iter_svo_poses.append(curr_pose)

    rmse = np.linalg.norm(curr_pose[:3, 3] - poses[curr_step][:3, 3])

    prev_pose = curr_pose
    prev_step = curr_step

In [ ]:
fig = plot_surface(map, showscale=False)
fig = plot_poses(poses[:END_FRAME], fig=fig, no_axes=True, color="black", name="Ground truth")
fig = plot_poses(svo_poses, fig=fig, no_axes=True, color="orange", name="VO poses")
fig = plot_poses(iter_svo_poses, fig=fig, no_axes=True, color="red", name="Iter VO poses", markersize=2)
# fig = plot_3d_points(landmark_points_cropped, fig=fig, color="red", markersize=2, name="Landmarks")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
fig.write_html("gtsam_slam.html")